In [29]:
# Common libraries
import matplotlib.pyplot as plt
from datetime import datetime
import requests
import json 
import pandas as pd

# API setup
from messari.messari import Messari                   # Messari
m = Messari('cb0d3a33-c4c5-4b50-9fff-5dce65d44b97')   # Messari
from pycoingecko import CoinGeckoAPI                  # Coingecko
cg = CoinGeckoAPI()                                   # Coingecko
API_KEY = '27mKdxqslM3aORzuco6LoqzKjFc'               # Glassnode

In [39]:
columns = ["Ticker", "PriceUSD", "Name", "Category1", "MarketCap", "TotalSupply", "CirculatingSupply", "TVL", "TotalAddresses", "ActiveAdresses", "Category2", "Category3"] 

In [40]:
master = pd.DataFrame(columns = columns)

In [49]:
symbols_df = pd.DataFrame(cg.get_coins_markets('usd'))
symbols = symbols_df['symbol']
master['Ticker'] = symbols
master.head()

,Ticker,Name,Category1,MarketCap,TotalSupply,CirculatingSupply,TVL,TotalAddresses,ActiveAdresses,Category2,Category3
0,btc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,eth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,usdt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
